<a href="https://colab.research.google.com/github/jlwollinger/river_level_prediction/blob/master/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

import json
import numpy as np
import cv2 as cv
import os
import matplotlib.pyplot as plt


def get_left_and_right_means(mask):
    means_left = []
    for i in range(0, len(mask), 10):
       intersection_points = []
       for j in range(len(mask[i])):
           if mask[i][j] == True:
               intersection_points.append(j)
               break
       if len(intersection_points) > 0: 
           means_left.append(sum(intersection_points) / len(intersection_points))
       else:
           means_left.append(0)
    
    means_right = []
    for i in range(0, len(mask), 10):
       intersection_points = []
       for j in range(len(mask[i]) - 1, 0, -1):
           if mask[i][j] == True:
               intersection_points.append(j)
               break
       if len(intersection_points) > 0: 
           means_right.append(sum(intersection_points) / len(intersection_points))
       else:
           means_left.append(0)
           
    means_left.extend(means_right)
    return means_left


  
X_train = np.array([])
y_train = np.array([])

annotations = json.load(open('annotations.json'))

values = list(annotations.values())
annotations = [a for a in values if a['regions']]

for value in annotations: 
    current_file = value
    y = float(current_file['filename'].split('_')[0])
    regions = current_file['regions']
    if regions != None and regions['0']['shape_attributes']['all_points_x'] != None:
        
        all_x = regions['0']['shape_attributes']['all_points_x']
    
        all_y = regions['0']['shape_attributes']['all_points_y']
    
        all_x = np.array(all_x)
        all_y = np.array(all_y)
    
        xy = np.dstack((all_x.ravel(), all_y.ravel()))[0]
    
    
        mask = np.zeros((1080, 1920))
        cv.fillPoly(mask, np.int32([xy]), 1)
        mask = mask.astype(bool)
    
        x = get_left_and_right_means(mask)
    
        X_train = np.append(X_train, x)
        y_train = np.append(y_train, y)
    
    
X_train = X_train.reshape((1291, 216, 1))


from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape=(X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)




Using TensorFlow backend.


Epoch 1/100
1291/1291 [==============================] - 20s 16ms/step - loss: 0.0553
Epoch 2/100
1291/1291 [==============================] - 20s 16ms/step - loss: 0.0191
Epoch 3/100
1291/1291 [==============================] - 22s 17ms/step - loss: 0.0127
Epoch 4/100
1291/1291 [==============================] - 22s 17ms/step - loss: 0.0119
Epoch 5/100
1291/1291 [==============================] - 21s 16ms/step - loss: 0.0111
Epoch 6/100
1291/1291 [==============================] - 21s 16ms/step - loss: 0.0109
Epoch 7/100
1291/1291 [==============================] - 21s 16ms/step - loss: 0.0101
Epoch 8/100
1291/1291 [==============================] - 21s 16ms/step - loss: 0.0111
Epoch 9/100
1291/1291 [==============================] - 21s 16ms/step - loss: 0.0102
Epoch 10/100
1291/1291 [==============================] - 21s 16ms/step - loss: 0.0097
Epoch 11/100
1291/1291 [==============================] - 21s 16ms/step - loss: 0.0094
Epoch 12/100
1291/1291 [============================

In [0]:

X_test = np.array([])
y_test = np.array([])

annotations = json.load(open("annotations_test.json"))

values = list(annotations.values())
annotations = [a for a in values if a['regions']]

for value in annotations: 
    current_file = value
    y = float(current_file['filename'].split('_')[0])
    regions = current_file['regions']
    if regions != None and regions['0']['shape_attributes']['all_points_x'] != None:
        
        all_x = regions['0']['shape_attributes']['all_points_x']
    
        all_y = regions['0']['shape_attributes']['all_points_y']
    
        all_x = np.array(all_x)
        all_y = np.array(all_y)
    
        xy = np.dstack((all_x.ravel(), all_y.ravel()))[0]

        mask = np.zeros((1080, 1920))
        cv.fillPoly(mask, np.int32([xy]), 1)
        mask = mask.astype(bool)

        x = get_left_and_right_means(mask)
    
        X_test = np.append(X_test, x)
        y_test = np.append(y_test, y)
    
    
X_test = X_test.reshape((144, 216))


predicted = regressor.predict(X_test)

